# Coupled Resonance Engine — Demo Analysis

This notebook demonstrates the full analytical pipeline from the white paper:

> **"Coupled Resonance Dynamics of Multi-Engine Rocket Clusters: A Cross-Scale Analytical Framework"** — G. Aharon, 2026

We reproduce all three main figures and explore custom parametric analysis.

**DISCLAIMER:** Analytical model — not experimentally validated. All outputs are for educational and research purposes only.

## 0. Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.dpi"] = 150

## 1. Engine & Cluster Configurations

CRE ships with pre-loaded SpaceX vehicle data from the white paper (Table 1).

In [ ]:
from cre.configs.engines import get_engine, list_engines
from cre.configs.clusters import get_cluster, list_clusters

print("Available engines:", list_engines())
print("Available clusters:", list_clusters())

In [ ]:
raptor = get_engine("raptor_2")
print(f"Engine: {raptor.name}")
print(f"  Thrust SL: {raptor.thrust_sl/1e3:.0f} kN")
print(f"  Chamber pressure: {raptor.chamber_pressure/1e5:.0f} bar")
print(f"  Speed of sound: {raptor.sound_speed} m/s")
print(f"  Injector: {raptor.injector_type}, Cycle: {raptor.cycle}")
print(f"  n range: {raptor.n_range}, tau range: {raptor.tau_range}")

In [ ]:
sh = get_cluster("super_heavy")
print(f"Cluster: {sh.name} ({sh.total_engines} engines)")
print(f"  Engine: {sh.engine_name}")
print(f"  Base diameter: {sh.base_diameter} m")
for i, ring in enumerate(sh.rings):
    print(f"  Ring {i}: N={ring.n_engines}, R={ring.radius}m, {ring.symmetry_group}")

## 2. Single-Engine Acoustics (Eqs 1–4)

Chamber acoustic modes and the Crocco combustion response.

In [ ]:
from cre.core.oscillator import chamber_acoustic_modes, nozzle_admittance

merlin = get_engine("merlin_1d")

for eng in [merlin, raptor]:
    modes = chamber_acoustic_modes(eng)
    Y = nozzle_admittance(eng)
    print(f"{eng.name}:")
    print(f"  1T = {modes.f_1T:.0f} Hz, 1L = {modes.f_1L:.0f} Hz, 2T = {modes.f_2T:.0f} Hz")
    print(f"  Nozzle admittance Y = {Y:.4e} s/m")

In [ ]:
from cre.core.crocco import crocco_magnitude, crocco_phase

omega = np.linspace(100, 20000, 1000) * 2 * np.pi  # 100 Hz to 20 kHz
freq = omega / (2 * np.pi)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 5), sharex=True)

for n_val, tau_val, label in [(1.0, 1e-3, "n=1, tau=1ms"),
                               (2.0, 1e-3, "n=2, tau=1ms"),
                               (1.0, 2e-3, "n=1, tau=2ms")]:
    mag = crocco_magnitude(omega, n_val, tau_val)
    phase = crocco_phase(omega, n_val, tau_val)
    ax1.plot(freq, mag, label=label)
    ax2.plot(freq, np.degrees(phase), label=label)

ax1.set_ylabel("|R(omega)|")
ax1.set_title("Crocco n-tau Combustion Response (Eq 2)")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)
ax2.set_xlabel("Frequency (Hz)")
ax2.set_ylabel("Phase (deg)")
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Coupled Mode Eigenfrequencies (Eqs 5–6)

Normal mode frequencies for N engines in a ring with inter-engine coupling kappa.

In [ ]:
from cre.core.coupled_modes import normal_mode_frequencies, mode_frequency_ratios

# Parameters representative of Raptor 2 on Super Heavy
k0 = 1e8       # Engine stiffness [N/m]
m = 1630.0      # Raptor 2 mass [kg]
kappa = 5e6     # Coupling coefficient [N/m]

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, N, title in zip(axes, [3, 10, 20],
                         ["Inner ring (N=3)", "Middle ring (N=10)", "Outer ring (N=20)"]):
    freqs = normal_mode_frequencies(k0, m, kappa, N) / (2 * np.pi)  # Hz
    ratios = mode_frequency_ratios(kappa, k0, N)
    ax.bar(range(N), freqs, color="steelblue", alpha=0.7)
    ax.set_xlabel("Mode index n")
    ax.set_ylabel("Frequency (Hz)")
    ax.set_title(title)
    ax.axhline(freqs[0], color="red", linestyle="--", alpha=0.5, label=f"Breathing: {freqs[0]:.0f} Hz")
    ax.legend(fontsize=7)
    ax.grid(True, alpha=0.3, axis="y")

plt.suptitle("Super Heavy: Normal Mode Frequencies per Ring (Eq 6)", fontsize=12)
plt.tight_layout()
plt.show()

## 4. Base Cavity Acoustics (Eqs 7–8)

Cavity mode frequencies scale with base diameter — larger vehicles have lower cavity modes.

In [ ]:
from cre.core.acoustics import cavity_mode_frequency, acoustic_transfer_function

# Speed of sound in hot recirculation zone
vehicles = {
    "Falcon 9": {"R": 3.66/2, "c": 843},
    "Super Heavy": {"R": 9.0/2, "c": 860},
}

for name, params in vehicles.items():
    f_1T = cavity_mode_frequency(params["c"], params["R"])
    print(f"{name}: f_1T = {f_1T:.1f} Hz (R={params['R']}m, c={params['c']}m/s)")

# Transfer function for Super Heavy base cavity
omega_sweep = np.linspace(10, 200, 500) * 2 * np.pi
f_1T_sh = cavity_mode_frequency(860, 9.0/2)
omega_1T = 2 * np.pi * f_1T_sh

H = acoustic_transfer_function(omega_sweep, g_i=1.0, g_j=0.8, omega_mn=omega_1T, Q_mn=15)

fig, ax = plt.subplots(figsize=(8, 4))
ax.semilogy(omega_sweep / (2*np.pi), np.abs(H))
ax.axvline(f_1T_sh, color="red", linestyle="--", alpha=0.5, label=f"f_1T = {f_1T_sh:.1f} Hz")
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("|H(omega)|")
ax.set_title("Super Heavy Base Cavity Transfer Function (Eq 7)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Figure 1 — Stability Boundaries in (n, tau) Space

Stability map showing critical interaction index vs. sensitive time lag.
The shaded region illustrates stability margin lost when transitioning from Earth to vacuum.

In [ ]:
from cre.core.stability import stability_boundary_sweep
from cre.plotting.stability_map import plot_stability_map

result_stab = stability_boundary_sweep(
    tau_range=(0.1e-3, 5.0e-3),
    frequencies=[50.0, 135.0, 56.0],
    alpha_earth=0.12,
    alpha_vacuum=0.06,
)

fig = plot_stability_map(result_stab)
plt.show()

## 6. Figure 2 — Damping Spectrum per Coupled Mode

The breathing mode (n=0) receives **no** inter-engine coupling damping — it is the most dangerous mode.
This is the central result of the white paper.

In [ ]:
from cre.configs.defaults import DEFAULT_DAMPING, EARTH_SL, LUNAR_VACUUM
from cre.core.damping import damping_spectrum_multi_env, breathing_mode_damping
from cre.plotting.damping_spectrum import plot_damping_spectrum

# Super Heavy outer ring (N=20)
ring = sh.rings[2]
result_damp = damping_spectrum_multi_env(ring.n_engines, DEFAULT_DAMPING, [EARTH_SL, LUNAR_VACUUM])

fig = plot_damping_spectrum(result_damp, zeta_crit=0.035)
plt.show()

# Highlight the breathing mode danger
zeta_earth = breathing_mode_damping(DEFAULT_DAMPING, EARTH_SL)
zeta_vacuum = breathing_mode_damping(DEFAULT_DAMPING, LUNAR_VACUUM)
print(f"Breathing mode (n=0) damping:")
print(f"  Earth: zeta = {zeta_earth:.4f}")
print(f"  Vacuum: zeta = {zeta_vacuum:.4f}")
print(f"  Margin lost in vacuum: {(1 - zeta_vacuum/zeta_earth)*100:.1f}%")

## 7. Figure 3 — Amplification Factor vs. Engine Count

Coherent oscillation scales as N (breathing mode), incoherent as sqrt(N).
Diamond markers show SpaceX vehicles.

In [ ]:
from cre.core.amplification import amplification_sweep
from cre.plotting.amplification import plot_amplification

result_amp = amplification_sweep(N_range=(1, 40), params=DEFAULT_DAMPING)

fig = plot_amplification(result_amp)
plt.show()

# Table 3 data
print("\nTable 3: Amplification factors")
print(f"{'N':>4s} {'Coherent':>10s} {'Incoherent':>12s} {'Ratio':>8s}")
print("-" * 38)
for N_val in [6, 9, 27, 33]:
    idx = np.where(result_amp.n_engines == N_val)[0][0]
    print(f"{N_val:4d} {result_amp.coherent[idx]:10.1f}x {result_amp.incoherent[idx]:12.2f}x {result_amp.ratio[idx]:8.2f}x")

## 8. Custom Analysis — What-If Scenarios

Explore how different parameters affect stability and damping.

### 8a. Varying atmospheric absorption

In [ ]:
from cre.core.stability import stability_boundary_sweep

fig, ax = plt.subplots(figsize=(8, 5))

tau = np.linspace(0.5e-3, 4e-3, 300)
colors = plt.cm.viridis(np.linspace(0, 1, 5))

for i, alpha_e in enumerate([0.06, 0.09, 0.12, 0.18, 0.24]):
    result = stability_boundary_sweep(
        tau_range=(0.5e-3, 4e-3),
        frequencies=[135.0],
        alpha_earth=alpha_e,
        alpha_vacuum=alpha_e * 0.5,
        n_tau=300,
    )
    ax.plot(result.tau * 1000, result.n_crit[0, 0, :],
            color=colors[i], linewidth=1.5,
            label=f"alpha={alpha_e:.2f} (Earth)")

ax.set_xlabel("tau (ms)")
ax.set_ylabel("n_crit")
ax.set_title("Stability Boundary Sensitivity to Absorption Coefficient")
ax.legend(fontsize=8)
ax.set_ylim(0, 5)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 8b. Comparing all SpaceX cluster damping spectra

In [ ]:
from cre.core.damping import damping_spectrum

cluster_names = ["falcon_9", "super_heavy", "starship"]

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, cname in zip(axes, cluster_names):
    cluster = get_cluster(cname)
    # Use the largest ring
    ring = max(cluster.rings, key=lambda r: r.n_engines)
    N = ring.n_engines

    zeta_earth = damping_spectrum(N, DEFAULT_DAMPING, EARTH_SL)
    zeta_vac = damping_spectrum(N, DEFAULT_DAMPING, LUNAR_VACUUM)

    x = np.arange(N)
    w = 0.35
    ax.bar(x - w/2, zeta_earth, w, label="Earth", color="#2196F3", alpha=0.7)
    ax.bar(x + w/2, zeta_vac, w, label="Vacuum", color="#F44336", alpha=0.7)
    ax.axhline(0.035, color="black", linestyle="--", linewidth=1, alpha=0.5)
    ax.set_xlabel("Mode n")
    ax.set_ylabel("zeta_total")
    ax.set_title(f"{cluster.name} (N={N})")
    ax.legend(fontsize=7)
    ax.grid(True, alpha=0.3, axis="y")

plt.suptitle("Damping Spectra Across SpaceX Vehicles", fontsize=12)
plt.tight_layout()
plt.show()

### 8c. Coupling coefficient sensitivity on mode spectrum

In [ ]:
from cre.core.coupled_modes import mode_frequency_ratios

N = 20  # Super Heavy outer ring
k0 = 1e8

fig, ax = plt.subplots(figsize=(8, 5))

kappa_values = [1e5, 5e5, 1e6, 5e6, 1e7]
colors = plt.cm.plasma(np.linspace(0.1, 0.9, len(kappa_values)))

for kappa, color in zip(kappa_values, colors):
    ratios = mode_frequency_ratios(kappa, k0, N)
    ax.plot(range(N), ratios, "o-", color=color, markersize=4,
            label=f"kappa/k0 = {kappa/k0:.1e}")

ax.set_xlabel("Mode index n")
ax.set_ylabel("omega_n / omega_0")
ax.set_title("Mode Frequency Spread vs. Coupling Strength (N=20)")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Summary

Key findings reproduced from the white paper:

1. **Breathing mode vulnerability**: Mode n=0 receives no inter-engine coupling damping in any environment.
2. **Vacuum amplifies risk**: Loss of atmospheric absorption reduces stability margins by ~40%.
3. **Coherent amplification**: N engines oscillating in phase produce N-times the perturbation of a single engine.
4. **Super Heavy**: 33-engine cluster with the largest breathing-mode risk among current vehicles.

---
*Analytical model — not experimentally validated.*